In [1]:
from scrape_scholar import *
import sqlite3 
import pandas as pd
import numpy as np
import networkx as nx
import altair as alt
import nx_altair as nxa
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
import pylab
from flask import Flask, render_template, request, redirect

In [10]:
def init_db():
    conn = sqlite3.connect("db.db")
    cur = conn.cursor()
 
    cur.executescript('''
    DROP TABLE IF EXISTS author;
    DROP TABLE IF EXISTS paper;
    DROP TABLE IF EXISTS author_paper;

    CREATE TABLE author(
    id INTEGER PRIMARY KEY NOT NULL,
    author TEXT NOT NULL UNIQUE);
    CREATE TABLE paper(
    id INTEGER PRIMARY KEY NOT NULL,
    paper VARCHAT NOT NULL UNIQUE);
    CREATE TABLE author_paper(
    id INTEGER PRIMARY KEY NOT NULL,
    author TEXT NOT NULL,
    paper TEXT NOT NULL);
    CREATE UNIQUE INDEX ind ON author_paper(author, paper);
    ''')

    conn.commit()
    conn.close()

In [11]:
init_db()

In [48]:
conn = sqlite3.connect("db.db")
cur = conn.cursor()

# Coletando os dados
data = scrape("Jorge Poco")

author_paper = pd.DataFrame(columns = ["paper", "author"])

for paper in range(len(data)):
    for author in data[paper].get("authors"):
        author_paper = author_paper.append({"paper" : data[paper].get("title"), "author" : author}, ignore_index = True)  
        
# Inserindo os dados
for paper in range(len(data)):
    conn.execute("INSERT OR IGNORE INTO paper (paper) VALUES (?)", [data[paper].get("title")])
    for author in data[paper].get("authors"):
        conn.execute("INSERT OR IGNORE INTO author (author) VALUES (?)", [author])
        
for index, rows in author_paper.iterrows(): 
    conn.execute("INSERT OR IGNORE INTO author_paper (author, paper) VALUES (?,?)", [rows.author, rows.paper])    
   
conn.commit()
conn.close()

In [49]:
conn = sqlite3.connect("db.db")
cur = conn.cursor()
inpt = pd.read_sql("SELECT * FROM author_paper", conn)
conn.close()

In [50]:
inpt

,id,author,paper
0,1,RS Targino,Sequential Monte Carlo samplers for capital al...
1,2,GW Peters,Sequential Monte Carlo samplers for capital al...
2,3,PV Shevchenko,Sequential Monte Carlo samplers for capital al...
3,4,GW Peters,Understanding operational risk capital approxi...
4,5,RS Targino,Understanding operational risk capital approxi...
5,6,PV Shevchenko,Understanding operational risk capital approxi...
6,7,RS Targino,Optimal exercise strategies for operational ri...
7,8,GW Peters,Optimal exercise strategies for operational ri...
8,9,G Sofronov,Optimal exercise strategies for operational ri...
9,10,PV Shevchenko,Optimal exercise strategies for operational ri...


In [ ]:
for paper in range(len(data)):
    for author in data[paper].get("authors"):
        author_paper = author_paper.append({"paper" : data[paper].get("title"), "author" : author}, ignore_index = True)  

try:    
    author_paper_old = pd.read_sql("SELECT author, paper FROM author_paper", conn)
    author_paper = pd.merge(author_paper_old, author_paper, how = "outer")

except:
    pass 

author_paper = pd.merge(pd.merge(author_paper, aux_author), aux_paper)
author_paper.to_sql("author_paper", con = conn, if_exists = "replace")

conn.close()

<hr>

In [ ]:
melhorar as rotas /author e /data, ver o que dá pra mudar